<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preamble" data-toc-modified-id="Preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preamble</a></span></li><li><span><a href="#Phase-Exinds" data-toc-modified-id="Phase-Exinds-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Phase Exinds</a></span></li><li><span><a href="#Exinds-for-Each-Phase-of-the-MJO" data-toc-modified-id="Exinds-for-Each-Phase-of-the-MJO-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exinds for Each Phase of the MJO</a></span></li><li><span><a href="#Pickle-1" data-toc-modified-id="Pickle-1-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Pickle 1</a></span></li><li><span><a href="#ENSO-Phases" data-toc-modified-id="ENSO-Phases-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>ENSO Phases</a></span></li><li><span><a href="#ENSO-Pase-Exinds" data-toc-modified-id="ENSO-Pase-Exinds-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>ENSO Pase Exinds</a></span></li><li><span><a href="#Pickle" data-toc-modified-id="Pickle-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Pickle</a></span></li></ul></div>

# Preamble

In [118]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import pickle

In [146]:
import warnings
warnings.filterwarnings("ignore")

In [119]:
path_2 = '/home/student.unimelb.edu.au/aborowiak/Desktop/Code/Current Project/Stations/MJO/MJO_df.pickle'
pickle_in_2 = open(path_2, 'rb')
MJO_df = pickle.load(pickle_in_2)

In [120]:
path = '/home/student.unimelb.edu.au/aborowiak/Desktop/Code/Current Project/Stations/Station_Daily_Rainfall_Data.pickle'
pickle_in = open(path, 'rb')
daily_rainfall_dict = pickle.load(pickle_in)

In [121]:
path = '/home/student.unimelb.edu.au/aborowiak/Desktop/Code/Current Project/Stations/MJO/RMM.pickle'
pickle_in = open(path, 'rb')
RMM = pickle.load(pickle_in)

# Phase Exinds

In [90]:
#Dictionary to store the collrections of the Two 
Rain_RMM_dict = {}


#Only want the MJO stations
current_stations = MJO_df.index


for station in current_stations:
    
    #Extracxting the raindata for an individual station
    rain_data = daily_rainfall_dict[station]
    
    #Getting the correct years for the period of study
    rain_data = rain_data.loc['1974-06-01':'2017']
    
    #Removing all the incorrect points
    rain_data = rain_data[rain_data["Rainfall (mm)"] < 8000]
    
    #Extracting only the days that are >1mm (Rainday)
    rain_data = rain_data[rain_data["Rainfall (mm)"] > 1]
    
    '''Combinging'''
    # Removing the bad points from the RMM
    RMM = RMM[RMM['Phase'] < 10]
    RMM = RMM[RMM['Amplitude'] < 10]
    
    
    #Making the RMM the same dates as the rain_data
    RMM_adjust = RMM[RMM.index.isin(rain_data.index)]
    
    
    Rain_RMM = pd.concat([rain_data, RMM_adjust], axis  = 1)
    
    
    Rain_RMM_dict[station] = Rain_RMM

# Exinds for Each Phase of the MJO

In [91]:
Rain_RMM_dict['4006'].head()

,Rainfall (mm),Phase,Amplitude
Date,,,
1974-07-29,5.0,4.0,0.796503
1975-01-09,3.0,4.0,1.411470
1975-01-10,3.0,5.0,1.382450
1975-01-30,12.0,8.0,2.389140
1975-02-01,34.0,1.0,2.026280


In [92]:
data = Rain_RMM_dict[station].groupby('Phase')['Rainfall (mm)'].size()
data

Phase
1.0    211
2.0    209
3.0    225
4.0    285
5.0    219
6.0    252
7.0    221
8.0    203
Name: Rainfall (mm), dtype: int64

In [93]:
data[2]

209

In [96]:
#Some DF's to store the results in:
Freq_df  = pd.DataFrame()
Int_df = pd.DataFrame()

current_stations = list(Rain_RMM_dict.keys())


for station in current_stations:
    'Frequency'
    
    phase_freqs = Rain_RMM_dict[station].groupby('Phase')['Rainfall (mm)'].size()
    
    Freq_df[station] = phase_freqs
    
    'Average Intensity'
    phase_ints = Rain_RMM_dict[station].groupby('Phase')['Rainfall (mm)'].mean()
    
    Int_df[station] = phase_ints

    
"Adjusting the INdex to be Phases of MJO"
mjo_phases = np.arange(1,9)

Freq_df['Phase'] = mjo_phases
Freq_df = Freq_df.set_index("Phase")

Int_df['Phase'] = mjo_phases
Int_df = Int_df.set_index("Phase")

In [97]:
Freq_df

,5004,5008,5014,4006,3028,3030,3024,3017,3027,2009,...,29048,37001,37049,36028,28004,31055,31011,32040,33073,39049
Phase,,,,,,,,,,,,,,,,,,,,,
1,112,86,95,100,74,97,115,178,138,168,...,167,126,120,108,257,693,608,228,158,211
2,78,76,95,143,110,130,191,265,185,226,...,196,142,143,118,287,772,656,298,200,209
3,68,92,132,196,141,155,210,284,223,277,...,223,184,155,128,310,723,638,324,206,225
4,89,114,145,169,139,161,204,302,234,299,...,228,171,166,155,360,751,675,333,246,285
5,125,140,144,182,201,234,278,357,281,313,...,264,169,170,148,407,707,627,355,252,219
6,96,145,153,172,208,228,289,334,286,356,...,299,166,172,141,468,675,597,345,278,252
7,101,120,153,162,168,207,246,312,258,330,...,307,167,169,148,487,652,602,354,291,221
8,69,73,100,128,113,147,183,257,175,235,...,204,155,132,115,339,542,480,281,229,203


# Pickle 1

In [109]:
MJO_normal_rain_dict = {'Frequency': Freq_df, 'Intensity (mm)': Int_df}

In [110]:
out_file = open('MJO_normal_rain_dict.pickle','wb')
pickle.dump(MJO_normal_rain_dict, out_file)
out_file.close()

# ENSO Phases

In [122]:
path = '/home/student.unimelb.edu.au/aborowiak/Desktop/Code/Current Project/Stations/ENSO/NINO34_wet_s.pickle'
pickle_in = open(path, 'rb')
NINO34_wet_s = pickle.load(pickle_in)

In [139]:
'''This code hear finds if Nino34 is above or below 0, a new column is created with a 1 if the wet season
is an El Nino event or -1 for La Nina'''

NINO_value = NINO34_wet_s.copy(deep = True)

NINO_value['Nino Value'] = np.where(NINO_value['Nino34'] > 0, 1, -1)

In [140]:
NINO_value.head()

,Nino34,Nino Value
Year,,
1871,-0.510000,-1
1872,-0.951667,-1
1873,-0.940000,-1
1874,-0.898333,-1
1875,-0.916667,-1


In [160]:
'''Dictionary'''

MJO_ENSO_normal_rain_dict = {}


current_stations = list(Rain_RMM_dict.keys())

for station in current_stations:
    RMM_rain = Rain_RMM_dict[station]
    
    '''ENSO years for EL Nino and La Nina'''
    #Finding the yers 
    
    El_nino = NINO_value[NINO_value['Nino Value'] == 1]
    El_nino_years = El_nino.index
    
    
    La_nina = NINO_value[NINO_value['Nino Value'] == -1]
    La_nina_years = La_nina.index
    
    #Creating two new DF's, one for EL Nino and one for La Nina, where there are only values that occured 
    #during years that where in El Nino or La Nina. A new column is created in the same style as above, 
    # where a 1 indicates an El Nino event occured durint that time, and -1 for La Nina.
    
    
    
    RMM_nino_rain = RMM_rain[RMM_rain.index.year.isin(El_nino_years)]
    RMM_nino_rain['Nino Value'] = np.tile([1], len(RMM_nino_rain))

    
    
    RMM_nina_rain = RMM_rain[RMM_rain.index.year.isin(La_nina_years)]
    RMM_nina_rain['Nino Value'] = np.tile([-1], len(RMM_nina_rain))
    
    
    '''Merging the two back together and then sorting them via dates (index)'''
    
    #Concatenating
    RMM_ENSO_rain = pd.concat([RMM_nino_rain,RMM_nina_rain], axis = 0)
    
    #Sorting
    RMM_ENSO_rain = RMM_ENSO_rain.sort_index()
    
    
    
    #Dictionary
    
    MJO_ENSO_normal_rain_dict[station] = RMM_ENSO_rain

In [164]:
MJO_ENSO_normal_rain_dict['2009'].head()

,Rainfall (mm),Phase,Amplitude,Nino Value
Date,,,,
1974-09-06,15.0,2.0,1.53587,-1
1974-09-07,1.8,2.0,1.70636,-1
1974-09-08,8.4,2.0,1.52284,-1
1974-09-09,4.0,2.0,1.27185,-1
1974-09-30,2.0,3.0,1.04540,-1


# ENSO Pase Exinds

In [ ]:
NINO_df_int = pd.DataFrame()
NINA_df_int = pd.DataFrame()


grouped_data_int = rain_data.groupby(['Phase', 'Nino Value']).mean()

# EL Nino
Nino_vals_int = grouped_data_int.loc[:,1]
NINO_df_int[station] = Nino_vals_int


#La Nina
Nina_vals_int = grouped_data_int.loc[:,-1]
NINA_df_int[station] = Nina_vals_int

In [309]:
MJO_stations = list(MJO_ENSO_normal_rain_dict.keys())

#DataFrame

NINO_df_freq = pd.DataFrame()
NINA_df_freq = pd.DataFrame()


NINO_df_int = pd.DataFrame()
NINA_df_int = pd.DataFrame()


for station in MJO_stations:
    
    #Extracting the Data
    rain_data = MJO_ENSO_normal_rain_dict[station]
    
    '''Frequency'''
    grouped_data_freq = rain_data.groupby(['Phase', 'Nino Value']).size() ######3
    
    # EL Nino
    Nino_vals_freq = grouped_data_freq.loc[:,1]
    NINO_df_freq[station] = Nino_vals_freq
    
    
    #La Nina
    Nina_vals_freq = grouped_data_freq.loc[:,-1]
    NINA_df_freq[station] = Nina_vals_freq
    
    '''Intensity'''

    grouped_data_int = rain_data.groupby(['Phase', 'Nino Value'])['Rainfall (mm)'].mean() ######3

    # EL Nino
    Nino_vals_int = grouped_data_int.loc[:,1]
    NINO_df_int[station] = Nino_vals_int


    #La Nina
    Nina_vals_int = grouped_data_int.loc[:,-1]
    NINA_df_int[station] = Nina_vals_int

    
    
'''Concatinating into A DataFrame'''
#Frequency
MJO_ENSO_normal_freq = pd.DataFrame()

MJO_ENSO_normal_freq = pd.concat({'El Nino': NINO_df_freq, 'La Nina':NINA_df_freq}, axis = 1)


#Intensity

MJO_ENSO_normal_int = pd.DataFrame()

MJO_ENSO_normal_int = pd.concat({'El Nino': NINO_df_int, 'La Nina':NINA_df_int}, axis = 1)




In [312]:
MJO_ENSO_normal_int

El Nino                                                         \
            5004       5008       5014       4006       3028       3030   
Phase                                                                     
1.0    15.550000   8.916667   9.512821  12.418919  11.804545  17.988889   
2.0     7.124138   9.956000  14.550000   8.811940   7.064286   8.829268   
3.0    11.692000  11.154545  16.230000   8.174468  13.521429  15.463768   
4.0    15.030769  18.970833  18.328358  14.489333  15.390141  20.639535   
5.0     9.944444  17.739394  20.246667  14.270930  21.027586  16.406306   
6.0    17.655814  13.028814  21.231818  15.824390  19.309302  22.454639   
7.0    16.874286  23.457778  18.677049  14.696610  16.064407  17.812857   
8.0     7.668182   7.726316  17.983871  12.576923  25.455000  19.189796   

                                                     ...        La Nina  \
            3024       3017       3027       2009    ...          29048   
Phase                                                ...                  
1.0    14.271795  11.460811  15.728261  12.035135    ...      20.428319   
2.0    13.010294   9.790083  12.963636  12.642424    ...      14.916379   
3.0    13.607955  14.277941  11.500000  12.968254    ...      17.167164   
4.0    17.902105  16.938776  15.030088  12.615385    ...      17.312500   
5.0    20.656667  18.653757  16.528676  15.607362    ...      17.442857   
6.0    20.943200  17.517986  20.527778  15.375887    ...      24.109314   
7.0    16.565476  15.210891  16.201235  13.102941    ...      20.110314   
8.0    17.932787  14.551961  11.975714  14.131313    ...      22.367717   

                                                                         \
           37001      37049      36028      28004      31055      31011   
Phase                                                                     
1.0    10.682432  11.187879  13.851562  13.970946  13.569287  14.477077   
2.0    17.230667  13.670930  13.833333  14.085207  12.829913  14.575131   
3.0    14.880556  13.710417  18.738462  11.998864  14.811346  15.300920   
4.0    11.418868  12.905941  18.122340  15.787000  15.005000  13.545082   
5.0    12.215094  10.303883  11.340964  14.933990  21.194531  20.166144   
6.0    12.777885  13.042308  13.618391  15.233333  22.893199  20.252486   
7.0    14.736458  15.303883  20.305376  18.165246  26.952162  23.540407   
8.0    10.451685  11.519737  11.938028  13.411351  22.275786  19.687455   

                                        
           32040      33073      39049  
Phase                                   
1.0    16.580147  16.855046  16.475556  
2.0    13.189474  12.896552  11.368939  
3.0    16.263536  17.890435  14.952593  
4.0    18.697238  19.714286  14.248352  
5.0    18.065128  25.655224  18.769718  
6.0    16.668636  17.453333  14.905917  
7.0    19.600426  20.652239  19.778082  
8.0    22.411932  20.400000  12.566949  

[8 rows x 64 columns]

# Pickle

In [313]:
MJO_ENSO_normal['Frequency']

El Nino                                               ...  La Nina  \
         5004 5008 5014 4006 3028 3030 3024 3017 3027 2009  ...    29048   
Phase                                                       ...            
1.0        42   36   39   37   22   27   39   74   46   74  ...      113   
2.0        29   25   42   67   28   41   68  121   77   99  ...      116   
3.0        25   33   60   94   56   69   88  136   86  126  ...      134   
4.0        39   48   67   75   71   86   95  147  113  156  ...      136   
5.0        45   66   60   86   87  111  120  173  136  163  ...      147   
6.0        43   59   66   82   86   97  125  139  126  141  ...      204   
7.0        35   45   61   59   59   70   84  101   81  136  ...      223   
8.0        22   19   31   52   40   49   61  102   70   99  ...      127   

                                                             
      37001 37049 36028 28004 31055 31011 32040 33073 39049  
Phase                                                        
1.0      74    66    64   148   407   349   136   109   135  
2.0      75    86    66   169   458   382   171   116   132  
3.0     108    96    78   176   379   326   181   115   135  
4.0     106   101    94   200   400   366   181   147   182  
5.0     106   103    83   203   384   319   195   134   142  
6.0     104   104    87   297   397   362   220   180   169  
7.0      96   103    93   305   370   344   235   201   146  
8.0      89    76    71   185   318   279   176   144   118  

[8 rows x 64 columns]

In [310]:
'''Dictionary'''

MJO_ENSO_normal = {'Frequency':MJO_ENSO_normal_freq, 'Intensity (mm)': MJO_ENSO_normal_int}

In [311]:
pickle_out = open('MJO_ENSO_normal.pickle','wb')
pickle.dump(MJO_ENSO_normal, pickle_out)
pickle_out.close()